<a href="https://colab.research.google.com/github/MeenTers/Project/blob/main/Render_View.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup pytorch 3D

In [1]:
import os
import sys
import torch
import numpy as np
import json
if torch.__version__=='1.6.0+cu101' and sys.platform.startswith('linux'):
    !pip install pytorch3d
else:
    need_pytorch3d=False
    try:
        import pytorch3d
    except ModuleNotFoundError:
        need_pytorch3d=True
    if need_pytorch3d:
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   128  100   128    0     0    677      0 --:--:-- --:--:-- --:--:--   677
100  404k    0  404k    0     0   328k      0 --:--:--  0:00:01 --:--:--  444k
  Cloning https://github.com/facebookresearch/pytorch3d.git (to revision stable) to /tmp/pip-req-build-oz9k2efc
  Running command git clone -q https://github.com/facebookresearch/pytorch3d.git /tmp/pip-req-build-oz9k2efc
  Running command git checkout -q 3fc3ea507bfbf7e1c2d5023a6db202698206f91d
     |████████████████████████████████| 50 kB 2.8 MB/s 
     |████████████████████████████████| 596 kB 11.0 MB/s 
  Created wheel for pytorch3d: filename=pytorch3d-0.6.1-cp37-cp37m-linux_x86_64.whl size=30325402 sha256=b61320366a9c65a11b60a3494e2e5d6276a34a3ff190ba1085dc9daa0b8e6aba
  Stored in directory: /tmp/pip-ephem-wheel-cache-5i5g6n4s/wheels/4c/03/f0/326bb241eb86a155a12f217c81e16

In [2]:
!pip install utils

In [3]:
!pip install Params

In [4]:
from PIL import Image 
import PIL

In [5]:
import matplotlib.pyplot as plt
from skimage.io import imread
from utils import *

# Util function for loading meshes
from pytorch3d.io import load_obj

# Data structures and functions for rendering
from pytorch3d.structures import Meshes
from pytorch3d.renderer import (
    look_at_view_transform,
    FoVPerspectiveCameras, 
    FoVOrthographicCameras, 
    Materials, 
    RasterizationSettings, 
    MeshRenderer, 
    MeshRasterizer,  
    SoftPhongShader,
    TexturesVertex,
    TexturesAtlas,
    PointsRenderer,
    PointsRasterizationSettings,
    PointsRasterizer
)

In [6]:
# Set our device:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")

In [ ]:
!wget -q https://github.com/Phonbopit/sarabun-webfont/raw/master/fonts/thsarabunnew-webfont.ttf

In [ ]:
# !pip install -U --pre matplotlib  
import matplotlib as mpl
mpl.font_manager.fontManager.addfont('thsarabunnew-webfont.ttf')
mpl.rc('font', family='TH Sarabun New')

In [7]:
def tensor_to_image(tensor):
    tensor = tensor*255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor)>3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return PIL.Image.fromarray(tensor)

In [450]:
!unzip '/content/drive/MyDrive/train_model/3D Model/Right/Most Damage.zip'

Archive:  /content/drive/MyDrive/train_model/3D Model/Right/Most Damage.zip
replace ID01.obj? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ID01.obj                
  inflating: ID02.obj                
  inflating: ID03.obj                
  inflating: ID04.obj                
  inflating: ID05.obj                
  inflating: ID07.obj                
  inflating: ID08.obj                


In [493]:
obj1 = '/content/ID08.obj'

In [494]:
# Get vertices, faces, and auxiliary information:
verts, faces, aux = load_obj(
    obj1,
    device=device,
    load_textures=True,
    create_texture_atlas=True,
    texture_atlas_size=4,
    texture_wrap="repeat"
     )

# Create a textures object
atlas = aux.texture_atlas

# Initialize the mesh with vertices, faces, and textures.
# Created Meshes object
capsule_mesh = Meshes(
    verts=[verts],
    faces=[faces.verts_idx],
    textures=TexturesAtlas(atlas=[atlas]),)

/usr/local/lib/python3.7/dist-packages/pytorch3d/io/obj_io.py:533: UserWarning: Mtl file does not exist: /content/ID08.mtl
  warnings.warn(f"Mtl file does not exist: {f}")


In [495]:
# Initialize the camera with camera distance, elevation, and azimuth angle
R, T = look_at_view_transform(dist = 0, elev = 
                              0, azim = 0) 
cameras = FoVPerspectiveCameras(device=device, R=R, T=T)

# Here we set the output image to be of size 256 x 256 based on config.json 
raster_settings = RasterizationSettings(
    image_size = 1024, 
    blur_radius = 0.0, 
    faces_per_pixel = 1, 
)

# Initialize rasterizer by using a MeshRasterizer class
rasterizer = MeshRasterizer(
        cameras=cameras, 
        raster_settings=raster_settings
    )

In [496]:
# The textured phong shader interpolates the texture uv coordinates for 
# each vertex, and samples from a texture image.
shader = SoftPhongShader(device = device, cameras = cameras)

# Create a mesh renderer by composing a rasterizer and a shader
renderer = MeshRenderer(rasterizer, shader)

In [497]:
# The batch size represents the number of different viewpoints from which we 
# want to render the mesh.
batch_size = 1

# Create a batch of meshes by repeating the capsule mesh and associated textures. 
# Meshes has a useful `extend` method which allows us do this very easily. 
meshes = capsule_mesh.extend(batch_size)

# Get a batch of viewing angles. 
elev = torch.linspace(0, 360,batch_size)
azim = [270]

# All the cameras helper methods support mixed type inputs and broadcasting. So we can 
# view the camera from the same distance and then specify elevation and azimuth angles 
# for each viewpoint as tensors. 
R, T = look_at_view_transform(dist = 5, elev = 0, azim = azim)
cameras = FoVPerspectiveCameras(device=device, R=R, T=T)
# We can pass arbirary keyword arguments to the rasterizer/shader via the renderer
# so the renderer does not need to be reinitialized if any of the settings change.
images = renderer(meshes, cameras=cameras)

In [498]:
save = []
for i in range(len(images)) :
 ims =  tensor_to_image(images[i].cpu().numpy())
 save.append(ims)

In [499]:
for i in range(len(save)) :
  save[i].save('/content/drive/MyDrive/train_model/Damage/Right/Most Damage/Test01/'+'ID08_'+str(3)+".png")